In [1]:
import pandas as pd
import pandas_market_calendars as mcal
import numpy as np

In [2]:
df = pd.read_csv("../data/preprocessed/base_dataset_news.csv")
df.head(2)

,Date,Headlines,Newspaper,XLE,XLF,XLK,XLV,XLY,positive,neutral,negative,finbert_label,finbert_confidence,avg_positive_global,avg_neutral_global,avg_negative_global,n_headlines,sent_index_global,is_trading_day
0,2018-03-20,"Cramer: Trump, the Fed and the cloud can get t...",cnbc,0,0,1,0,0,0.381990,0.606314,0.011696,neutral,0.606314,0.186842,0.466275,0.346883,81,-0.160042,1
1,2018-03-20,G20 financial leaders say need more dialogue o...,reuters,0,0,0,0,0,0.259833,0.077557,0.662610,negative,0.662610,0.186842,0.466275,0.346883,81,-0.160042,1


In [3]:
df_agg = (
    df
    .groupby("Date", as_index=False)
    .agg({
        "avg_positive_global": "first",
        "avg_neutral_global": "first",
        "avg_negative_global": "first",
        "sent_index_global": "first",
        "n_headlines": "first",
        "is_trading_day": "first",
    }))

df_agg.head()

,Date,avg_positive_global,avg_neutral_global,avg_negative_global,sent_index_global,n_headlines,is_trading_day
0,2018-03-20,0.186842,0.466275,0.346883,-0.160042,81,1
1,2018-03-21,0.176286,0.497613,0.326101,-0.149815,87,1
2,2018-03-22,0.193766,0.449283,0.356951,-0.163184,100,1
3,2018-03-23,0.221206,0.412813,0.365981,-0.144774,79,1
4,2018-03-24,0.127403,0.536494,0.336103,-0.208700,11,0


In [4]:
columns = ['Date', 'avg_positive_global', 'avg_neutral_global', 'avg_negative_global', 'n_headlines', 'sent_index_global', 'is_trading_day']

df_agg = df_agg[columns]
df_agg.head(2)

,Date,avg_positive_global,avg_neutral_global,avg_negative_global,n_headlines,sent_index_global,is_trading_day
0,2018-03-20,0.186842,0.466275,0.346883,81,-0.160042,1
1,2018-03-21,0.176286,0.497613,0.326101,87,-0.149815,1


In [5]:
df_agg["Date"] = pd.to_datetime(df_agg["Date"])
df_agg = df_agg.sort_values("Date")
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 851 non-null    datetime64[ns]
 1   avg_positive_global  851 non-null    float64       
 2   avg_neutral_global   851 non-null    float64       
 3   avg_negative_global  851 non-null    float64       
 4   n_headlines          851 non-null    int64         
 5   sent_index_global    851 non-null    float64       
 6   is_trading_day       851 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(2)
memory usage: 46.7 KB


In [6]:
df_agg.to_csv("../data/preprocessed/base_trading_days.csv", index=False)

In [7]:
df_agg["TradingDate"] = df_agg["Date"].where(df_agg["is_trading_day"] == 1)
# df_agg.to_csv("test.csv")

df_agg["TradingDate"] = df_agg["TradingDate"].bfill() # backward fill but the name is confusing, looking for the next trading day


# In case there are trailing non-trading days after the last trading day, drop 
# df_agg = df_agg.dropna(subset=["TradingDate"])

In [8]:
df_agg.head(10)

,Date,avg_positive_global,avg_neutral_global,avg_negative_global,n_headlines,sent_index_global,is_trading_day,TradingDate
0,2018-03-20,0.186842,0.466275,0.346883,81,-0.160042,1,2018-03-20
1,2018-03-21,0.176286,0.497613,0.326101,87,-0.149815,1,2018-03-21
2,2018-03-22,0.193766,0.449283,0.356951,100,-0.163184,1,2018-03-22
3,2018-03-23,0.221206,0.412813,0.365981,79,-0.144774,1,2018-03-23
4,2018-03-24,0.127403,0.536494,0.336103,11,-0.208700,0,2018-03-26
5,2018-03-25,0.173876,0.520802,0.305322,38,-0.131446,0,2018-03-26
6,2018-03-26,0.205676,0.447310,0.347014,80,-0.141337,1,2018-03-26
7,2018-03-27,0.196626,0.475401,0.327973,87,-0.131347,1,2018-03-27
8,2018-03-28,0.224163,0.510148,0.265689,85,-0.041526,1,2018-03-28
9,2018-03-29,0.149836,0.502540,0.347624,83,-0.197788,1,2018-03-29


In [9]:
df_agg.columns

Index(['Date', 'avg_positive_global', 'avg_neutral_global',
       'avg_negative_global', 'n_headlines', 'sent_index_global',
       'is_trading_day', 'TradingDate'],
      dtype='object')

In [10]:
df_daily = (
    df_agg.groupby("TradingDate", as_index=False).agg({
        "avg_positive_global": "mean",
        "avg_neutral_global": "mean",
        "avg_negative_global": "mean",
        "n_headlines": "sum",
        "sent_index_global": "mean"
    })
    .rename(columns={"TradingDate": "Date"})
    .sort_values("Date")
    .reset_index(drop=True)
)

df_daily["is_trading_day"] = 1

In [11]:
df_daily.head()

,Date,avg_positive_global,avg_neutral_global,avg_negative_global,n_headlines,sent_index_global,is_trading_day
0,2018-03-20,0.186842,0.466275,0.346883,81,-0.160042,1
1,2018-03-21,0.176286,0.497613,0.326101,87,-0.149815,1
2,2018-03-22,0.193766,0.449283,0.356951,100,-0.163184,1
3,2018-03-23,0.221206,0.412813,0.365981,79,-0.144774,1
4,2018-03-26,0.168985,0.501535,0.329480,129,-0.160494,1


In [12]:
df_daily.to_csv("../data/preprocessed/base_trading_days_backfill.csv", index=False)